In [1]:
import os

import llama_index.core

os.environ['OPENAI_API_KEY'] = ''
os.environ['PINECONE_API_KEY'] = ''
openai_api_key = os.environ['OPENAI_API_KEY']
pinecone_api_key = os.environ['PINECONE_API_KEY']

In [2]:
from datasets import load_dataset

dataset = load_dataset("lcw99/wikipedia-korean-20221001", split='train[:100]')

In [3]:
data = dataset.to_pandas()[['id', 'text', 'title']]
data.drop_duplicates(subset='text', keep='first', inplace=True)
data.head()

,id,text,title
0,5,"제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대...",지미 카터
1,9,"수학(數學, , 줄여서 math)은 수, 양, 구조, 공간, 변화 등의 개념을 다루...",수학
2,10,"수학에서 상수란 그 값이 변하지 않는 불변량으로, 변수의 반대말이다. 물리 상수와는...",수학 상수
3,19,"문학(文學, )은 언어를 예술적 표현의 제재로 삼아 새로운 의미를 창출하여, 인간과...",문학
4,20,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,나라 목록


In [4]:
from llama_index.core import Document

docs = []

for i, row in data.iterrows():
    docs.append(Document(
        text=row['text'],
        doc_id=row['id'],
        extra_info={'title': row['title']}
    ))

docs[0]

Document(id_='5', embedding=None, metadata={'title': '지미 카터'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.\n\n생애\n\n어린 시절 \n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n\n정계 입문 \n1962년 조지아주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n\n대통령 재임 \n\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받고 제럴드 포드 대통령을 누르고 당선되었다.\n\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.\n\n외교 정책 \n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n\n카터는 1970년대 후반 당시 대한민국 등 인권 후진

In [5]:
# 노드로 청킹
from llama_index.core.node_parser import SimpleNodeParser

parser = SimpleNodeParser()

nodes = parser.get_nodes_from_documents(docs)
nodes[0]

TextNode(id_='0b8ee7d8-65e7-4354-937b-7be3b5a1328c', embedding=None, metadata={'title': '지미 카터'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='5', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'title': '지미 카터'}, hash='6c4f6250b2c71196e0876bb59e94d9496a6fae62ee2104a452c19434f7e2bc0d'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='7edfa881-7e6d-4777-8897-e279f2c7c262', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='b1c0fcd751e6cd8d42a1d9900d9c841da7c7ff46b5583259d4b56955547c75f6')}, text='제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.\n\n생애\n\n어린 시절 \n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n\n정계 입문 \n1962년 조지아주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 

In [6]:
# pincone 연결
from pinecone import Pinecone, PodSpec

api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

In [7]:
from time import sleep

index_name = 'vectordbprac'
dimension = 1536
metric = 'dotproduct'
spec = PodSpec('gcp-starter')

if index_name in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.delete_index(index_name)

pc.create_index(index_name, dimension=dimension, metric=metric, spec=spec)

while not pc.describe_index(index_name).status['ready']:
    sleep(1)

index = pc.Index(index_name)

index_stats = index.describe_index_stats()
print(index_stats)

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


In [9]:
from llama_index.vector_stores.pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(pinecone_index=index)

In [10]:
from llama_index.core import VectorStoreIndex, StorageContext, ServiceContext, Settings
from llama_index.embeddings.openai import OpenAIEmbedding

# 벡터스토어 인덱스에 들어가는 스토리지 컴포넌트 정의
storage_context = StorageContext.from_defaults(
    vector_store=vector_store
)

# 서비스 컨텍스트에 해당 인덱스를 활용한 연결 컴포넌트 정의
embed_model = OpenAIEmbedding(model='text-embedding-3-small', embed_batch_size=100)
service_context = ServiceContext.from_defaults(embed_model=embed_model)

index = VectorStoreIndex.from_documents(
    docs, storage_context=storage_context, service_context=service_context
)


/var/folders/yk/wkvyrshx24sdfg84jy4w_xnc0000gn/T/ipykernel_4638/3882198299.py:11: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embed_model)


Upserted vectors:   0%|          | 0/516 [00:00<?, ?it/s]

In [13]:
query_engine = index.as_query_engine()
res = query_engine.query('지미 카터가 누구야?')
print(res)

지미 카터는 미국의 39대 대통령으로, 민주당 출신이며 1977년부터 1981년까지 대통령으로 임명되었습니다. 그는 대통령으로서 중동 평화를 위한 캠프데이비드 협정을 이끌었고, 소련과의 제2차 전략 무기 제한 협상(SALT II)에 참여했습니다. 또한 인권 문제와 도덕정치를 강조하며 대한민국과의 관계에서도 중요한 영향을 미쳤습니다.
